### Import packages that may need to be used in the future. 

In [1]:
import requests
import lxml as lx
from bs4 import BeautifulSoup
import time
from json import loads
import pandas

### Generate an access token that lasts for an hour. 

In [15]:
client_id = "21d31ff6fc1f41fea5b03cf985160b80"
client_secret = "b73e2df24c354dd69819e20cb7da5b86"

token_request_url = "https://accounts.spotify.com/api/token"
token_request_headers = {"Content-Type": "application/x-www-form-urlencoded"}
token_request_data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

token_request_response = requests.post(url=token_request_url, headers=token_request_headers, data=token_request_data)

if token_request_response.status_code == 200:
    token_info = token_request_response.json()
    print(token_info)  # This will contain the access token
else:
    print(f"Error: {token_request_response.status_code}, {token_request_response.text}")

{'access_token': 'BQASJWj6IHjnm_M6xEHqcde6i3IQ-7nVRA3j6SMJ8FjJ1_Wbm7ihCIum8aAbJLK9hqi44JqERcctyJN_gXypiEHMUW-fkL2qNAhrKFtTE0xd6ezprpfdYQpDH-ayYxT-a7L15zq24tY', 'token_type': 'Bearer', 'expires_in': 3600}


In [17]:
access_token = token_info.get('access_token')

print(access_token)

BQASJWj6IHjnm_M6xEHqcde6i3IQ-7nVRA3j6SMJ8FjJ1_Wbm7ihCIum8aAbJLK9hqi44JqERcctyJN_gXypiEHMUW-fkL2qNAhrKFtTE0xd6ezprpfdYQpDH-ayYxT-a7L15zq24tY


### Function to get the artist's info given a link to the artist. 

In [27]:
def get_artist_info(artist_url):
    url = artist_url
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        artist_data = response.json()
        print(artist_data)  # Prints the artist's details
    else:
        print(f"Error: {response.status_code}, {response.text}")

get_artist_info("https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ") # The Weeknd https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ?si=HhB2mjsUSQ-5oYN7A9i6ZA
get_artist_info("https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s") # Justin Bieber https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s?si=qXJYi_CdTri2kj_3d1_HHg


{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'}, 'followers': {'href': None, 'total': 101372361}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ', 'id': '1Xyo4u8uXC1ZmMpatF05PJ', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb9e528993a2820267b97f6aae', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051749e528993a2820267b97f6aae', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1789e528993a2820267b97f6aae', 'height': 160, 'width': 160}], 'name': 'The Weeknd', 'popularity': 97, 'type': 'artist', 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'followers': {'href': None, 'total': 81222845}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'id': '1uNFoZAHBGtllmzznpCI3s', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5e

### Function to obtain all data on a playlist. 

In [41]:
def get_playlist_info(playlist_url):
    url = playlist_url
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        playlist_data = response.json()
        print(playlist_data)  # Prints the playlist's details
    else:
        print(f"Error: {response.status_code}, {response.text}")

    """
    Save song 'name','release_date', 'release_date_precision', 'id', 'external_ids', artist 'name',  'id', 'popularity'
    """

get_playlist_info("https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ") # https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ?si=b65f446ee0174c78

{'collaborative': False, 'description': 'The official <a href="http://www.billboard.com/charts/hot-100">Billboard Hot 100</a> features this week\'s most popular songs across all genres, ranked by radio airplay monitored by Nielsen BDS, download sales tracked by Nielsen SoundScan and streaming activity data provided by leading online music services. For more information go to <a href="http://Billboard.com">Billboard.com</a>.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ'}, 'followers': {'href': None, 'total': 1804606}, 'href': 'https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ', 'id': '6UeSakyzhiEt4NB3UAd6NQ', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c3bd5501a335b265807df34db', 'width': None}], 'name': 'Billboard Hot 100', 'owner': {'display_name': 'Billboard', 'external_urls': {'spotify': 'https://open.spotify.com/user/billboard.com'}, 'href': 'https://api.spotify.com/v1/users/billb

### Function that gets the useful information of each track on a playlist. 

In [33]:
def get_playlist_tracks_info(playlist_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(playlist_url, headers=headers)
    
    if response.status_code == 200:
        playlist_data = response.json()
        
        extracted_tracks = []
        
        for item in playlist_data.get('tracks', {}).get('items', []):
            track = item.get('track', {})
            
            if track:
                track_info = {
                    "song_name": track.get('name'),
                    "song_id": track.get('id'),
                    "release_date": track.get('album', {}).get('release_date'),
                    "release_date_precision": track.get('album', {}).get('release_date_precision'),
                    "external_ids": track.get('external_ids', {}),
                    "artists": [
                        {
                            "artist_name": artist.get('name'),
                            "artist_id": artist.get('id'),
                            "popularity": track.get('popularity')
                        }
                        for artist in track.get('artists', [])
                    ]
                }
                extracted_tracks.append(track_info)
        
        return extracted_tracks

    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

playlist_url = "https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ"
playlist_info = get_playlist_tracks_info(playlist_url, access_token)

if playlist_info:
    for track in playlist_info:
        print(track)

{'song_name': 'luther (with sza)', 'song_id': '45J4avUb9Ni0bnETYaYFVJ', 'release_date': '2024-11-22', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUG12408496'}, 'artists': [{'artist_name': 'Kendrick Lamar', 'artist_id': '2YZyLoL8N0Wb9xBt1NhZWg', 'popularity': 92}, {'artist_name': 'SZA', 'artist_id': '7tYKF4w9nC0nq9CsPZTHyP', 'popularity': 92}]}
{'song_name': 'Die With A Smile', 'song_id': '2plbrEY59IikOBgBGLjaoe', 'release_date': '2024-08-16', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUM72409273'}, 'artists': [{'artist_name': 'Lady Gaga', 'artist_id': '1HY2Jd0NmPuamShAr6KMms', 'popularity': 100}, {'artist_name': 'Bruno Mars', 'artist_id': '0du5cEVh5yTK9QJze8zA0C', 'popularity': 100}]}
{'song_name': 'Not Like Us', 'song_id': '6AI3ezQ4o3HUoP6Dhudph3', 'release_date': '2024-05-04', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUG12400910'}, 'artists': [{'artist_name': 'Kendrick Lamar', 'artist_id': '2YZyLoL8N0Wb9xBt1NhZWg', 'popularity': 9